In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

In [0]:
import pandas as pd


prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None) # re-name test.csv 
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})

# display 
test_df.head()

In [0]:
!pip install allennlp
!pip install nltk
!pip install tqdm

!pip install -U spacy 
!python -m spacy download en_core_web_sm

In [0]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")

START AGAIN

In [0]:
def extract_modal(obj):
    arr = list() # full set of constituents
    modals= list() # only the modal phrases
    pasts = list() # only the past phrases
    nouns = list()
    verbs = list()

    def extract(obj, arr):
        
        if isinstance(obj, dict):
          for k, v in obj.items():
            if isinstance(v, (dict, list)):
                extract(v, arr)
              
            if k=='nodeType' and v=='NP':
                arr.append(v)
            if k=='nodeType' and v=='MD':
                arr.append(v)
            if k=='nodeType' and v=='VP':    
                arr.append(v)
            if k=='nodeType' and v=='VBN': # Verb, past participle
                arr.append(v)    
            if k=='nodeType' and v=='VBD': # Verb, past tense
                arr.append(v)
            if k=='nodeType' and v=='VBP': # Verb, past tense, not 3rd person singular
                arr.append(v)     
            if k=='nodeType' and v=='ADVP': # Adverb Phrase
                arr.append(v)           
            if k=='nodeType' and v=='RB':   # Adverb 
                arr.append(v)
            if k=='nodeType' and v=='CC':   # Coordinating conjunction
                arr.append(v)
            if k=='nodeType' and v=='IN':   # Preposition or subordinating conjunction 
                arr.append(v)        
            if k=='nodeType' and v=='SBAR': # Clause starting subordinating conjunction
                arr.append(v)
            if k=='nodeType' and v=='SINV': # Inverted declarative sentence, i.e. one in which the subject follows the tensed verb or modal.
                arr.append(v)

        elif isinstance(obj, list):
          prev = obj[0]
          for item in obj:
            if isinstance(item, dict):
              #print(item['word'])
              if ('would'==item['word'].lower() or 'could'==item['word'].lower() or
                  'wouldn\'t'==item['word'].lower() or 'could\'t'==item['word'].lower() or
                  'woulnd\'t'==item['word'].lower() or 'coulnd\'t'==item['word'].lower() or
                  'might'==item['word'].lower() or 'should'==item['word'].lower() or
                  'mightn\'t'==item['word'].lower() or 'shouldn\'t'==item['word'].lower() or
                  'mighn\'t'==item['word'].lower() or 'shoulnd\'t'==item['word'].lower() or
                  'd'==item['word'].lower() or item['word'].lower().strip().endswith('\'d') or
                  item['word'].lower().lstrip().startswith('as if ') or
                  item['word'].lower().lstrip().startswith('as though ')
              ):
                phrase = scrap_from(obj, item)
                modals.append({'word':item['word'],'text':phrase}) # phrase starting from item

              if 'CC' in arr: 
                arr.remove('CC')   
                if ( 'but' == prev['word'].lower() or
                  'because' == prev['word'].lower() or
                  'although' == prev['word'].lower()
                ):
                  phrase = scrap_from(obj, item)
                  modals.append({'word':item['word'],'text':phrase}) # phrase starting from item

              if 'MD' in arr: 
                arr.remove('MD')
                for h in obj: # head word
                  if h['word'].lower().strip()=='\'d':
                     modals.append({'word':h['word'],'text':obj})
                  if h['word'].lower().lstrip().startswith('would'):
                     modals.append({'word':h['word'],'text':obj})
                  if h['word'].lower().lstrip().startswith('could'):
                     modals.append({'word':h['word'],'text':obj})
                  if h['word'].lower().lstrip().startswith('should'):
                     modals.append({'word':h['word'],'text':obj})
                  if h['word'].lower().lstrip().startswith('may'):
                     modals.append({'word':h['word'],'text':obj})   
                  if h['word'].lower().lstrip().startswith('might'):
                     modals.append({'word':h['word'],'text':obj}) 
                  if h['word'].lower().lstrip().startswith('ought'):
                     modals.append({'word':h['word'],'text':obj})    

              if 'VBP' in arr: # common misspelling of modals
                arr.remove('VBP')
                if prev['word'].lower().lstrip().startswith('wouln'):
                    modals.append({'word':prev['word'],'text':obj})
                if prev['word'].lower().lstrip().startswith('couln'):
                    modals.append({'word':prev['word'],'text':obj})
                if prev['word'].lower().lstrip().startswith('shouln'): # 
                    modals.append({'word':prev['word'],'text':obj})
                if prev['word'].lower().lstrip().startswith('mighn'):
                    modals.append({'word':prev['word'],'text':obj}) 
                if prev['word'].lower().lstrip().startswith('oughn'):
                    modals.append({'word':prev['word'],'text':obj})  

              if 'NP' in arr: 
                arr.remove('NP')
                nouns.append({'word':'NP','text':obj})  

              if 'SINV' in arr: # Inverted declarative sentence (eg Had it been...)
                arr.remove('SINV')
                pasts.append({'word':'SINV','text':obj}) 

              if 'VP' in arr:
                arr.remove('VP')
                verbs.append({'word':'VP','text':obj})
                if 'wish' == prev['word'].lower():
                  pasts.append({'word':prev['word'],'text':obj})  
                if 'should' in item['word'].lower():
                  phrase = scrap_from(obj, item)
                  pasts.append({'word':prev['word'],'text':phrase})   

              if 'VBD' in arr: 
                arr.remove('VBD')
                if 'were'==prev['word'].lower():
                  pasts.append({'word':prev['word'],'text':obj})                  
                if 'was'== prev['word'].lower():
                  pasts.append({'word':prev['word'],'text':obj})  
                if 'had'== prev['word'].lower():
                  pasts.append({'word':prev['word'],'text':obj})

              # Constituent parser mistakes 'VBN' for 'VBD'(past)
              if 'VBN' in arr: 
                arr.remove('VBN')
                if 'had'== prev['word'].lower():
                  pasts.append({'word':prev['word'],'text':obj}) 
                  
              if 'RB' in arr: 
                arr.remove('RB')
                if 'even' == prev['word'].lower():
                  pasts.append({'word':prev['word'],'text':obj})
                     
              if 'IN' in arr: 
                arr.remove('IN') 
                if ('in' == prev['word'].lower() or
                    'if' == prev['word'].lower() or
                    'with' == prev['word'].lower() or
                    'without'== prev['word'].lower() or
                    'unless'== prev['word'].lower()
                ):    
                  phrase = scrap_from(obj, item)
                  pasts.append({'word':item['word'],'text':phrase})     
                
              #arr.append(item) # uncomment for the full set
              #print(item)

              extract(item, arr)
              prev = item
        return arr

    results = extract(obj, arr)

    return modals, pasts, nouns, verbs

In [0]:
def scrap_from(obj, item): 
  word = str() 
  prev = str() 
  for h in obj: # head word
    if h['word'] == item['word']: 
      word = ' ' 
    if len(word) > 0: 
      word = word.strip() 
      if (not (
              word.lower()=='\'m' or word.lower()=='\'ll' or
              word.lower()=='\'s' or word.lower()=='d' or # I'd be...
              word.lower()=='\'re' or word.lower()=='\'ve' or word.lower()=='n\'t') and  
          not (prev.lower().endswith('s') and word.lower()=='\' ') and # before\':peoples'
          not (word.lower() in '+.: m k x' and prev[-1:].isnumeric()) and # eg:'10:30am', '3M'
          not (prev.lower() in '+' and word.isnumeric()) and
          not (
              prev.lower() in '¥$€£&@({[/---#'  or  # after(eg 'S& P')
              word.lower() in '^~*:&%)}]/---...,;' # before(eg '% 5 M')
          )
      ): 
        word = word + ' '
      word = word + h['word']  
      prev = word
  word = normalize(word)     
  #print(word)    
  return word

In [0]:
def separate_children(obj):
    arr = list()

    def separate(obj, arr):
        
        if isinstance(obj, dict):

            for k, v in obj.items():
              if k != 'children':
                if isinstance(v, (dict, list)):
                    separate(v, arr)
                elif k == 'word':      
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                separate(item, arr)
         
        return arr

    results = separate(obj, arr)
    return results

In [0]:
def extract_constituents(prediction):
 for k, v in prediction.items():
  if (k=='hierplane_tree'):
    for ke, val in v.items():
      if (ke=='root'):
        for key, value in val.items():
          if (key == 'children'):
            return extract_modal(value)  

In [0]:
def deduplicate(past):
  for p1 in past:
    for p2 in past:
      if p2 in p1 and len(p1) > len(p2):
        if p2 in past:
          past.remove(p2)          
  return past         

In [0]:
def remove_from(past, modal, last=1):
  pst = past.copy()

  for m in sorted(modal,key=len): # in order of longer consequents
    for p in reversed(sorted(past,key=len)): # shorter antecedents
      if p.rstrip() in m and len(m) > len(p): # p is substring of m
        if p in past and len(past) > last: 
          past.remove(p)
      elif m.rstrip() in p and len(p) > len(m):# m is substring of p
        if m in modal and len(modal) > last: # can't be left empty!
          modal.remove(m) 

  for p in pst: # fresh copy
    for m in modal:
      if m.rstrip() in p: # both p = m or p > m
        if p in past and len(past) > last:
          past.remove(p) # remove p{ m }

  return past, modal         

In [0]:
def separate_from(md,pa):
  if pa in md and len(md) > len(pa):
    if md.index(pa) > 0 :
      md = md[0:md.index(pa)]
      #print('|'+md+'{...}')
    else:
      md = md.replace(pa,'')
      #print('|{...}'+md)  
  return md        

In [0]:
from nltk.tokenize import word_tokenize

def simplify(constit):
  phrase  = constit['word']
  
  constit = constit['text']
  if isinstance(constit, str):
    word = normalize(constit)
    return word
    
  gist = list();
  unique = set() 
  for c in constit:
      p = separate_children(c) # list of c['word']s
      for head in sorted(p,key=len):
        #print(head) # head phrase (few words or one char ',')
        tokenized = word_tokenize(head)
        tokens = set(tokenized) #  without duplicates (eg ',')
        if len(unique.intersection(tokens)) <= len(tokens):
          unique.update(tokens) # without duplicate tokens!!
          gist.extend(tokenized)
  #print(gist)     
  result = str()
  prev = str()
  for word in gist:
    word = word.strip()
    if (
        not (
             word.lower()=='\'m' or word.lower()=='\'ll' or 
             word.lower()=='\'s' or word.lower()=='d' or # I'd be doing...
             word.lower()=='\'re' or word.lower()=='\'ve' or word=='n\'t'
        ) and
        not (prev.lower().endswith('s') and word.lower()=='\' ') and # before\':peoples'
        not (word.lower() in '+.: m k x' and prev[-1:].isnumeric()) and # after '10:30am'
        not (prev.lower() in '+' and word.isnumeric()) and
        not (prev.lower()=='i' and word.lower()=='d') and # ID - identification
        not  
        (
          prev.lower() in '¥$€£&@({[/---#' or  # ' ' after
          word.lower() in '^~*:&%)}]/---...,;'# or before
        )
    ): 
      result = result + ' '
      #print(prev,'-->',word)
    prev = word
    result = result + word 
  result = normalize(result)   
  #print(phrase,':',result)  
  return result

In [0]:
import re
regex = '\\\'\s+([^\']+)\s+\\\''
def compact(fragment):
# specify the number of replacements by changing the 4th argument
  result = re.sub(regex, '\''+r'\1'+'\'' , fragment)
  return result

In [0]:
def normalize(fragment):
  r = compact(fragment).strip()
  r = r.replace('   ',' ') 
  r = r.replace('  ',' ')
  r = r.replace(' ,',',') 
  r = r.replace(' .','.')
  r = r.replace('Mr.','Mr. ')
  r = r.replace('Ms.','Ms. ')
  r = r.replace('Mrs.','Mrs. ')
  r = r.replace('.  ','. ') # Mr./Ms./Mrs.
  r = r.replace('... ','...')
  r = r.replace(' --- ','---')
  r = r.replace(' -- ','--')
  r = r.replace(' - ','-')
  r = r.replace(' !','!')
  r = r.replace(' ?','?')
  r = r.replace('$ ','$')
  r = r.replace('# ','#')
  r = r.replace(' %','%')
  r = r.replace('( ','(')
  r = r.replace(' )',')')
  r = r.replace(' ;',';')
  r = r.replace(' :',':') 
  r = r.replace(': ',':') #50/50 - carefull!
  r = r.replace(' nt','nt')
  r = r.replace(' (k)','(k)') #401(k)
  r = r.replace(' (tm)','(tm)')
  r = r.replace('n na ','nna ')
  r = r.replace(' \'m ','\'m ')
  r = r.replace('s \' ','s\' ') # was 'neurtralized'?
  r = r.replace(' \'s ','\'s ') # a 'surgical' strike!
  r = r.replace(' \'d ','\'d ')
  r = r.replace(' \'re ','\'re ')
  r = r.replace(' \'ve ','\'ve ')
  r = r.replace(' \'ll ','\'ll ')
  r = r.replace(' n\'t','n\'t') # what about UPPER()?
  r = r.replace(' \' ',' \'')
  r = r.rstrip()
  return r

In [0]:
def defragment(fragments, text, original, removed):
  result = []
  candid = []
  defrag = []
  prev = str() # previous fragment
  start = 0 # start of next fragment
  end = -1 # end of previous fragment  
  for f in sorted(fragments,key=len):
    r = normalize(f)
    #print(prev+' | '+r)
    try: 
      start = text.index(r)
    except ValueError:
      #print('\n{'+r+'} not in:\n '+text) # this is now in offset()
      pass
    if start > end: # assume no overlap with prev (ie separte fragment)
      defrag = prev + r # try to join two fragments
      index = text.find(defrag) # is it exactly consequitive fragment?
      if index < 0 or index > end: # it separate and non-consequitive 
        end = start + len(r) - 1
        prev = r
        if end < len(text):
          candid.append(overlay(start, {'labels':0, 'text':r, 'start':start, 'end':end}, original, removed))
      elif index > 0 and index+len(prev)+len(r) < len(text): # consequitive
        start = index 
        end = start+len(prev)+len(r)-1
        prev = text[start:end]
        candid.append(overlay(start, {'labels':0, 'text':defrag, 'start':start, 'end':end}, original, removed))
    elif start < end: # next fragment is located before prev end
      defrag = r + prev
      index = text.find(defrag) # is it exactly preceding prev fragment?
      #print(text,index,defrag)
      if index > 0 and index+len(prev)+len(r)-1 == end:
        start = index
        prev = text[start:end]
        candid.append(overlay(start, {'labels':0, 'text':defrag, 'start':start, 'end':end}, original, removed))
      elif start >= 0:
        end = start + len(r) - 1
        prev = r
        if end < len(text):
          candid.append(overlay(start, {'labels':0, 'text':r, 'start':start, 'end':end}, original, removed))
        
  if len(candid) == 1:
    candid[0].update(labels = 1) # gold  

  result.extend(candid)

  return result       

In [0]:
def overlay(starting, fragment, original, removed):
  assert isinstance(fragment, dict)

  base = offset(fragment['text'], starting, original)
  text = base[0]
  start= base[1]
  end = base[2]
  #print(start,text,end)

  for r in removed:
    #print(r[0],start,r[1],end)
    if r[0] <= start:
      start=start + 1
      end = end + 1
      #print(start,text,end)
    elif r[0] <= end:
      end = end + 1
      #print(start,text,end) 
    
  fragment.update(text = text)  
  fragment.update(start=start) 
  fragment.update(end = end) 

  return fragment

In [0]:
def offset(text, starting, original):
  start = -1
  end = -1
  prev = str()
  for c in text:
    try: 
      #print(c, start, prev, end)
      start = original.index(prev + c, starting)
      end = start + len(prev) + len(c) - 1
    except ValueError:
      if c==' ': # eg.'Pfizer (NYSE: '+'PFE)'
        last = original.find(prev, starting)
        if last > -1: # back 1 space helped
          prev = prev # + c
          continue    # skip 1 space
        else:   
          if end >= 0: # reverse from end
            end = end + 1 # 1 space
            start = end - len(prev)
            text = original[start:end]
            #print(start, prev, end, text)
          else:
            print('\n\"'+prev+'\" not from '+str(starting) + ' in:'+original)
          return text, start, end  
      else: 
        last = original.find(prev + ' ' + c, starting)
        if last > -1: # add 1 space 
          prev = prev + ' ' + c
          continue
        else:  
          last = original.find(prev+c)
          if last == -1: # eg 'bla-' instead of 'bla -'
            last = original.find(prev) # leave 'bla'
          if last > -1: # without space!
            start = last
            end = start + len (prev) + len(c) 
            prev = prev + c
            continue
          if start >= 0: # reverse from start
            end = start + len(prev) + len(c)
            text = original[start:end]
            #print(c,start,prev,end,text)
          else:
            print('\n\"'+prev+c+'\" not from '+str(starting) + ' in:'+original)  
          return text,start,end 

    prev = prev + c

  #print(start, prev, end, text)  
  return text, start, end  

In [0]:
def record(original):
  base=[(i, c) for i, c in enumerate(original)]
  return base

In [0]:
def contract(text, base):
  removed = []
  i=0
  for b in base: 
    if text.find(b[1],b[0]-i)==-1:
      removed.append(b)
      i = i + 1
  return removed

In [0]:
def chunk(original):
  verbs = list()
  modal = list()
  nounm = list()
  past = list() 
  nounp = list()
  nouns = list()

  base = record(original)
  original = original.replace('\"','') # hack the ""
  removed = contract(original, base)
  
  text = normalize(original) # remove extra spaces

  prediction = predictor.predict(sentence=text)
  mod, pas, nou, ver = extract_constituents(prediction)

  for v in ver:
    vr = simplify(v) 
    if vr not in verbs:
      verbs.append(vr)

  for u in nou:
    nu = simplify(u) 
    if nu not in nouns:
      nouns.append(nu) 

  if len(mod)==0: # if no modal('MD') detected 
    for p in pas:
      pa = simplify(p)
      if pa not in past:
          past.append(pa) 

  for m in reversed(sorted(mod,key=len)):
    md = simplify(m) # get one string gist
    if len(md)==0:
      continue
    #print('=======================')  
    #print(md)
    
    for nm in reversed(sorted(nouns,key=len)):  
      if nm in md:   
        nounm.append(nm)
        nouns.remove(nm)

    for p in reversed(sorted(pas,key=len)):
      pa = simplify(p)
      if len(pa)==0:
        continue
      #print('-----------------------')
      #print(pa)

      # consider join first, but how?
      new = separate_from(md, pa)
      if new != md and len(new) > 0:
        if md in modal: # remove original
            modal.remove(md) 
        md = new 

      new = separate_from(pa, md)
      if new != pa and len(new) > 0:
        if pa in past: # remove original
            past.remove(pa) 
        pa = new 

      # separate NPs originating with past tense      
      for np in reversed(sorted(nouns,key=len)):
        if np in pa:
          nounp.append(np)
          nouns.remove(np)
        
          #new = separate_from(pa, np)
          #if new != pa and len(new) > 0:
          #  if pa in past: # remove original
          #      past.remove(pa) 
          #  pa = new 
        
      if pa not in past and len(pa) > 0:
        past.append(pa)         

    if md not in modal and len(md) > 0:        
      modal.append(md)
      
  result = remove_from(modal, past)
  modal = result[0]
  past = result[1]
 
  # follwoing does not affect modal, past
  nounm = remove_from(nounm, modal,1)[0]
  nounm = remove_from(nounm, verbs,0)[0]
  nounp = remove_from(nounp, past, 0)[0]
  nounp = remove_from(nounp, verbs,0)[0]

  # only after nouns are removed above
  modal = remove_from(modal, nounm)[0]
  past = remove_from(past, nounp)[0]

  verbs = deduplicate(verbs)
  nounm = deduplicate(nounm)
  nounp = deduplicate(nounp)
  modal = deduplicate(modal)
  #past = deduplicate(past)

  # get start, end positions 
  modals = defragment(modal, text, original, removed)
  #if len(modals)==0:
  #  modals = defragment(verbs, text, original, removed)

  pasts = defragment(past, text, original, removed)
  if len(pasts)==0:
    if len(modals) > 1:
      pasts.append(modals.pop(0))
      if len(modals) == 1:
         modals[0].update(labels=1) # gold  
    else:  
      pasts = defragment(nouns, text, original, removed)

  return modals, pasts, verbs, nouns, nounm, nounp

In [0]:
import nltk
nltk.download('punkt')

modals, pasts, verbs, nouns, nounm, nounp = chunk(
  original = "Reading between the lines: Judge Stearns used some choice descriptions for the Cape Wind opponents, calling Cape Wind's opponent's obdurate and saying that while they typically claim the mantle of environmentalism, the opponents in this case have doffed their green garb and draped themselves in the banner of free-market economics. One part of the complaint, Stearns said, referenced documents that contradict on their face a supposed fact as plead to make an argument that is misleading and ultimately untrue. In a footnote, Stearns also wrote that were he an economist or a conservationist instead of a judge, he may have rejected Cape Wind, but that elected officials, regulators and the courts have reviewed and approved the project."    
)

print(nouns)
print(verbs)
print(nounm)
print(nounp)
print(pasts)
print(modals)     

In [0]:
import nltk
nltk.download('punkt')

import pandas as pd

from tqdm import tqdm
from time import sleep

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None) # re-name test.csv 
test_df = test_df.drop(index=0)
  
test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1], 
    'labels': ''
})

count = test_df['id'].count()
print(count)

i = 0
j = 0
modal_df= pd.DataFrame(columns=['id','labels','text','start', 'end'])
past_df = pd.DataFrame(columns=['id','labels','text','start', 'end'])
simple_df = pd.DataFrame(columns=['id','labels','text'])

for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  oid = test_df['id'][ind]

  modals, pasts, verbs, nouns, nounm, nounp = chunk(original)

  for m in modals:
    m_row = pd.Series(data={
      'id': test_df['id'][ind]+'C'+ str(i),
      'labels': m['labels'],
      'text': m['text'] ,
      'start': m['start'],
      'end': m['end']
    }, name = str(i) )
    modal_df = modal_df.append(m_row)
    i = i+1
 
  for p in pasts:
    p_row = pd.Series(data={
      'id': test_df['id'][ind]+'A'+ str(j),
      'labels': p['labels'],
      'text': p['text'] ,
      'start': p['start'],
      'end': p['end']
    }, name = str(j) )
    past_df = past_df.append(p_row)   
    j = j+1   

  who = str()
  for n in nounp:
    who = who  +  n + ' / '

  what = str()
  for nn in nounm:
    what = what + ' / ' + nn

  i = 0
  j = 0
  n = 0
  for m in modals:
    if len(pasts) > 0:
      for p in pasts:
        labels = str(int(m['labels'])*int(p['labels']))
        simple = who + p['text'] + ' | ' + m['text'] + what
        #print(labels, simple)
        s_row = pd.Series(data={
          'id': test_df['id'][ind]+'A'+ str(j) + 'C'+ str(i),
          'labels': labels,
          'text': simple
        }, name = str(n) )
        simple_df = simple_df.append(s_row) 
        j = j+1 
        n = n+1
    else:
      s_row = pd.Series(data={
        'id': test_df['id'][ind]+'A'+ str(j) + 'C'+ str(i),
        'labels':m['labels'],
        'text': who + ' | ' + m['text'] + what
      }, name = str(n) )
      simple_df = simple_df.append(s_row) 
    i = i+1
    n = n+1

  sleep(1)  

simple_df.head(100)

In [0]:
past_df.to_csv(prefix+'antecedents.tsv', sep='\t', index=False, header=False)
modal_df.to_csv(prefix+'consequents.tsv', sep='\t', index=False, header=False)
simple_df.to_csv(prefix+'simplified.tsv', sep='\t', index=False, header=False)
simple_df.head()

In [0]:
!mv /content/antecedents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/consequents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/simplified.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/